In [1]:
# Imports

%reload_ext autoreload
%autoreload 3

import torch
from torch import nn
import pytorch_lightning as pl

from src.cogponder import CogPonderModel
from src.cogponder.data import CogPonderDataModule
from src.cogponder.data.stroop_sro import StroopSRODataset


In [39]:
# Experiment

step_duration = 10  # in ms
batch_size = 512

dataset = StroopSRODataset(response_step_interval=step_duration)

datamodule = CogPonderDataModule(dataset, batch_size=batch_size, num_workers=8)

configs = {
    'inputs_dim': dataset[:][3].size(1),
    'outputs_dim': torch.unique(dataset[:][4]).size(0),
    'embeddings_dim': 8,
    'response_loss_beta': 1.,
    'time_loss_beta': 10.,
    'learning_rate': 1e-2,
    'max_response_step': dataset[:][5].max().int().item() + 10,
    'n_contexts': torch.unique(dataset[:][2]).size(0),
    'n_subjects': torch.unique(dataset[:][1]).size(0),
    'subject_embeddings_dim': 2,
    'task': 'stroop'
}

model = CogPonderModel(**configs)

trainer = pl.Trainer(max_epochs=1000, accelerator='cpu', log_every_n_steps=2)

trainer.fit(model, datamodule=datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name               | Type                 | Params
------------------------------------------------------------
0 | operator_node      | SimpleOperatorModule | 108   
1 | halt_node          | HaltingModule        | 81    
2 | recurrence_node    | RecurrenceModule     | 336   
3 | subject_embeddings | Embedding            | 1.0 K 
4 | embeddings         | Embedding            | 16    
5 | resp_loss_fn       | ResponseLoss         | 0     
6 | time_loss_fn       | ResponseTimeLoss     | 0     
------------------------------------------------------------
1.6 K     Trainable params
0         Non-trainable params
1.6 K     Total params
0.006     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/Users/morteza/micromamba/envs/cogponder/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
